In [1]:
!pip install transformers
!pip install tensorflow
from transformers import BertTokenizer
import tensorflow as tf
import numpy as np

In [2]:
# Initialize the GPT-2 tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

eos_token = tokenizer.sep_token_id
pad_token = tokenizer.pad_token_id

In [3]:
input_sequences = []
target_tokens = []
max_length = 64

with open("dataset.txt", "r", encoding="utf-8") as file:
    for line in file:
        question, answer = line.strip().split('\t')

        # Encode the texts. Adding the EOS token is important for GPT to know when a sentence ends
        question_tokens = tokenizer.encode(question, add_special_tokens=False)
        answer_tokens = tokenizer.encode(answer, add_special_tokens=False) + [eos_token] # Ensure EOS token is added to the answer

        # Construct sequences of max_length or shorter
        for i in range(0, len(answer_tokens)):
            # Input sequence: question + end token + part of the answer
            seq = question_tokens + [eos_token] + answer_tokens[:i]
            if len(seq) > max_length:
                break

            # Ensure sequences are padded to have uniform length
            input_sequences.append(seq + [pad_token] * (max_length - len(seq)))  # Padding

            # Target token is the next token in the answer
            target_tokens.append(answer_tokens[i])

In [8]:
for seq in input_sequences[:10]:
    #print(seq)
    print(tokenizer.decode(np.array(seq), clean_up_tokenization_spaces=True))
    print('\n')
          

hi, how are you doing? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


hi, how are you doing? [SEP] i [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


hi, how are you doing? [SEP] i'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [9]:
for seq in target_tokens[:10]:
    print(tokenizer.decode(np.array(seq), clean_up_tokenization_spaces=True))

i
'
m
f i n e
.
h o w
a b o u t
y o u r s e l f
?
[ S E P ]


In [10]:
# convert to TensorFlow tensors
input_sequences = tf.constant(input_sequences, dtype=tf.int16)
target_tokens = tf.constant(target_tokens, dtype=tf.int16)

In [11]:
# add dataset to 
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_tokens))
dataset = dataset.shuffle(buffer_size=1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)